# Flight Delay Prediction - Data Exploration

This notebook explores the airline on-time performance data and provides initial insights into flight delay patterns.

## Objectives
1. Download and load airline data from Bureau of Transportation Statistics
2. Perform initial data exploration and quality assessment
3. Analyze delay patterns by various dimensions
4. Identify key factors influencing flight delays
5. Prepare data for feature engineering

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
from pathlib import Path

# Add src to path
sys.path.append('../src')

from data.download_data import download_airline_data, load_airline_data
from visualization.plots import FlightDelayVisualizer

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")

%matplotlib inline

## 1. Data Download and Loading

In [ ]:
# Download sample data (first 3 months of 2023)
print("Downloading airline data...")
download_airline_data(year=2023, months=[1, 2, 3])

In [ ]:
# Load the data
print("Loading airline data...")
df = load_airline_data(year=2023, sample_size=100000)  # Load 100k sample for exploration

if df is not None:
    print(f"Loaded {len(df):,} rows and {len(df.columns)} columns")
    display(df.head())
else:
    print("Failed to load data. Please check the download process.")

## 2. Data Quality Assessment

In [ ]:
# Basic data info
print("Dataset Shape:", df.shape)
print("\nColumn Data Types:")
print(df.dtypes)

print("\nMemory Usage:")
print(f"{df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# Missing values analysis
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})

missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Percentage', ascending=False)

print("Missing Values Summary:")
display(missing_df.head(15))

# Visualize missing data
if len(missing_df) > 0:
    plt.figure(figsize=(12, 6))
    missing_df.head(15)['Missing Percentage'].plot(kind='bar')
    plt.title('Missing Data by Column')
    plt.ylabel('Missing Percentage (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Key columns for analysis
key_columns = ['FL_DATE', 'OP_CARRIER', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'CRS_ARR_TIME',
               'DEP_DELAY', 'ARR_DELAY', 'CANCELLED', 'DIVERTED', 'DISTANCE']

available_columns = [col for col in key_columns if col in df.columns]
print(f"Available key columns: {available_columns}")

# Basic statistics for key numerical columns
numerical_cols = ['DEP_DELAY', 'ARR_DELAY', 'DISTANCE']
available_numerical = [col for col in numerical_cols if col in df.columns]

if available_numerical:
    print("\nBasic Statistics for Key Numerical Columns:")
    display(df[available_numerical].describe())

## 3. Target Variable Analysis

In [ ]:
# Create target variable (delayed > 15 minutes)
if 'ARR_DELAY' in df.columns:
    df['delayed'] = (df['ARR_DELAY'] > 15).astype(int)
    
    # Remove cancelled and diverted flights for main analysis
    if 'CANCELLED' in df.columns:
        df_analysis = df[(df['CANCELLED'] != 1) & (df['ARR_DELAY'].notna())].copy()
    else:
        df_analysis = df[df['ARR_DELAY'].notna()].copy()
    
    print(f"Analysis dataset: {len(df_analysis):,} flights")
    
    # Target variable distribution
    delay_counts = df_analysis['delayed'].value_counts()
    delay_pct = df_analysis['delayed'].value_counts(normalize=True) * 100
    
    print("\nTarget Variable Distribution:")
    print(f"On-time (≤15 min delay): {delay_counts[0]:,} ({delay_pct[0]:.1f}%)")
    print(f"Delayed (>15 min): {delay_counts[1]:,} ({delay_pct[1]:.1f}%)")
    
    # Initialize visualizer
    viz = FlightDelayVisualizer()
    
    # Plot delay distribution
    viz.plot_delay_distribution(df_analysis)
else:
    print("ARR_DELAY column not found in dataset")

## 4. Temporal Analysis

In [ ]:
# Convert date and create time features
if 'FL_DATE' in df_analysis.columns:
    df_analysis['FL_DATE'] = pd.to_datetime(df_analysis['FL_DATE'])
    df_analysis['month'] = df_analysis['FL_DATE'].dt.month
    df_analysis['day_of_week'] = df_analysis['FL_DATE'].dt.dayofweek
    df_analysis['day_of_month'] = df_analysis['FL_DATE'].dt.day
    
    if 'CRS_DEP_TIME' in df_analysis.columns:
        df_analysis['dep_hour'] = (df_analysis['CRS_DEP_TIME'] // 100).fillna(12)
    
    # Plot temporal patterns
    viz.plot_delays_by_time(df_analysis)

## 5. Airport and Route Analysis

In [ ]:
# Top airports by volume
if 'ORIGIN' in df_analysis.columns and 'DEST' in df_analysis.columns:
    print("Top 15 Origin Airports by Flight Volume:")
    origin_volume = df_analysis['ORIGIN'].value_counts().head(15)
    display(origin_volume)
    
    print("\nTop 15 Destination Airports by Flight Volume:")
    dest_volume = df_analysis['DEST'].value_counts().head(15)
    display(dest_volume)
    
    # Plot airport delay patterns
    viz.plot_airport_delays(df_analysis, top_n=15)

In [ ]:
# Route analysis
if 'ORIGIN' in df_analysis.columns and 'DEST' in df_analysis.columns:
    df_analysis['route'] = df_analysis['ORIGIN'] + '_' + df_analysis['DEST']
    
    # Top routes by volume
    print("Top 15 Routes by Flight Volume:")
    route_volume = df_analysis['route'].value_counts().head(15)
    display(route_volume)
    
    # Top routes by delay rate (minimum 100 flights)
    route_delays = df_analysis.groupby('route').agg({
        'delayed': ['count', 'mean']
    }).round(3)
    
    route_delays.columns = ['flight_count', 'delay_rate']
    route_delays = route_delays[route_delays['flight_count'] >= 100]
    
    print("\nTop 15 Routes by Delay Rate (min 100 flights):")
    display(route_delays.sort_values('delay_rate', ascending=False).head(15))

## 6. Airline Analysis

In [ ]:
# Airline performance analysis
if 'OP_CARRIER' in df_analysis.columns:
    airline_stats = df_analysis.groupby('OP_CARRIER').agg({
        'delayed': ['count', 'mean'],
        'ARR_DELAY': ['mean', 'median', 'std']
    }).round(2)
    
    airline_stats.columns = ['flight_count', 'delay_rate', 'avg_delay', 'median_delay', 'delay_std']
    airline_stats = airline_stats[airline_stats['flight_count'] >= 1000]  # Minimum 1000 flights
    
    print("Airline Performance Summary (min 1000 flights):")
    display(airline_stats.sort_values('delay_rate', ascending=False))
    
    # Visualize airline performance
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    airline_stats['delay_rate'].plot(kind='bar', color='skyblue')
    plt.title('Delay Rate by Airline')
    plt.ylabel('Delay Rate')
    plt.xticks(rotation=45)
    
    plt.subplot(2, 2, 2)
    airline_stats['avg_delay'].plot(kind='bar', color='lightcoral')
    plt.title('Average Delay by Airline')
    plt.ylabel('Average Delay (minutes)')
    plt.xticks(rotation=45)
    
    plt.subplot(2, 2, 3)
    airline_stats['flight_count'].plot(kind='bar', color='lightgreen')
    plt.title('Flight Volume by Airline')
    plt.ylabel('Number of Flights')
    plt.xticks(rotation=45)
    
    plt.subplot(2, 2, 4)
    plt.scatter(airline_stats['flight_count'], airline_stats['delay_rate'], 
               s=100, alpha=0.7, color='purple')
    plt.xlabel('Flight Count')
    plt.ylabel('Delay Rate')
    plt.title('Flight Volume vs Delay Rate')
    
    # Add airline labels
    for idx, row in airline_stats.iterrows():
        plt.annotate(idx, (row['flight_count'], row['delay_rate']), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.tight_layout()
    plt.show()

## 7. Distance and Flight Duration Analysis

In [ ]:
# Distance analysis
if 'DISTANCE' in df_analysis.columns:
    # Create distance categories
    df_analysis['distance_category'] = pd.cut(df_analysis['DISTANCE'], 
                                            bins=[0, 500, 1000, 1500, 2000, float('inf')],
                                            labels=['Short (<500mi)', 'Medium (500-1000mi)', 
                                                   'Long (1000-1500mi)', 'Very Long (1500-2000mi)', 
                                                   'Ultra Long (>2000mi)'])
    
    distance_analysis = df_analysis.groupby('distance_category').agg({
        'delayed': ['count', 'mean'],
        'ARR_DELAY': 'mean'
    }).round(3)
    
    distance_analysis.columns = ['flight_count', 'delay_rate', 'avg_delay']
    
    print("Flight Performance by Distance Category:")
    display(distance_analysis)
    
    # Plot distance vs delays
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    distance_analysis['delay_rate'].plot(kind='bar', color='orange')
    plt.title('Delay Rate by Distance Category')
    plt.ylabel('Delay Rate')
    plt.xticks(rotation=45)
    
    plt.subplot(1, 2, 2)
    plt.scatter(df_analysis['DISTANCE'], df_analysis['ARR_DELAY'], 
               alpha=0.1, s=1, color='red')
    plt.xlabel('Distance (miles)')
    plt.ylabel('Arrival Delay (minutes)')
    plt.title('Distance vs Arrival Delay')
    plt.ylim(-50, 200)  # Limit y-axis for better visualization
    
    plt.tight_layout()
    plt.show()

## 8. Correlation Analysis

In [ ]:
# Select numerical columns for correlation analysis
numerical_columns = df_analysis.select_dtypes(include=[np.number]).columns
correlation_cols = [col for col in numerical_columns if col not in ['Unnamed: 0']]

if len(correlation_cols) > 1:
    # Calculate correlation with target variable
    target_correlations = df_analysis[correlation_cols].corr()['delayed'].abs().sort_values(ascending=False)
    
    print("Top correlations with delay target:")
    display(target_correlations.head(15))
    
    # Create correlation heatmap
    viz.plot_correlation_heatmap(df_analysis[correlation_cols[:15]])  # Top 15 features

## 9. Data Quality and Preparation Summary

In [ ]:
# Summary of data quality issues and recommendations
print("=== DATA QUALITY SUMMARY ===")
print(f"Total flights in dataset: {len(df):,}")
print(f"Flights for analysis (non-cancelled, with arrival data): {len(df_analysis):,}")
print(f"Data completeness: {len(df_analysis)/len(df)*100:.1f}%")

if 'delayed' in df_analysis.columns:
    print(f"\nTarget variable (delayed > 15 min):")
    print(f"- Delayed flights: {df_analysis['delayed'].sum():,} ({df_analysis['delayed'].mean()*100:.1f}%)")
    print(f"- On-time flights: {(df_analysis['delayed']==0).sum():,} ({(1-df_analysis['delayed'].mean())*100:.1f}%)")
    print(f"- Class imbalance ratio: {(df_analysis['delayed']==0).sum() / df_analysis['delayed'].sum():.1f}:1")

print("\n=== KEY FINDINGS ===")
print("1. Class imbalance: Most flights are on-time (need to handle in modeling)")
print("2. Temporal patterns: Clear variation by hour, day of week, and month")
print("3. Airport effects: Significant variation in delay rates by airport")
print("4. Airline differences: Substantial variation in performance between carriers")
print("5. Route specificity: Some routes consistently have higher delay rates")

print("\n=== NEXT STEPS ===")
print("1. Integrate weather data for enhanced predictions")
print("2. Engineer features for aircraft lag effects")
print("3. Create airport congestion metrics")
print("4. Build time-based cyclical features")
print("5. Handle class imbalance in modeling")

In [ ]:
# Save processed data for next notebook
output_path = '../data/processed/airline_exploration.csv'
df_analysis.to_csv(output_path, index=False)
print(f"Saved processed data to {output_path}")
print(f"Shape: {df_analysis.shape}")